# Enade - Transformação de dados - parte 01

In [ ]:
#importando bibliotecas necessárias
import pandas as pd
import numpy as np
import os

In [ ]:
enade = pd.read_csv(
    "./enade2019/microdados_enade_2019/2019/3.DADOS/microdados_enade_2019.txt",
    sep=';', decimal=','

)

In [ ]:
enade.head()

In [ ]:
#verificando schema da tabela
dict(enade.dtypes)


- CO_IES
- CO_CATEGAD
- CO_GRUPO
- CO_MODALIDADE
- CO_UF_CURSO
- CO_REGIAO_CURSO
- NU_IDADE
- TP_SEXO
- NT_GER
- NT_FG
- NT_CE

MAIS ALGUNS ITENS DO QUESTIONÁRIO DO ESTUDANTE:
- 01: Estado Civil
- 02: cor ou raça
- 04: Escolaridade do pai
- 05: Escolaridade da mãe
- 08: Renda Familiar
- 10: Situação de trabalho
- 11: Situação de bolsas
- 14: Intercâmbio
- 15: Cotas
- 23: Horas de estudo por semana
- 25: Motivo de escolha do curso
- 26: Motivo de escolha da IES
    

In [ ]:
#Alguns dados da coluna NT_GER
enade.NT_GER.describe()

In [ ]:
# contando numeros de nulos
enade.NT_GER.isnull().sum()

In [ ]:
# quantidade relativa de nulos
enade.NT_GER.isnull().sum() / enade.shape[0] #divisao pela quantidade total de linhas (posição 0 da tupla)

In [ ]:
# acessando estatísticas descritivas da nota geral
# filtrando alunos da região nordeste (cod 2 de acordo com o dicionário de variáveis)
enade.loc[
    
    enade.CO_REGIAO_CURSO == 2
].NT_GER.describe()

In [ ]:
#calculando estatísticas descritivas apenas para aluno com nota maior que zero
enade.loc[
    enade.NT_GER > 0
].NT_GER.describe()

In [ ]:
# filtrando apenas alunos entre 20 e 50 anos
enade.loc[
    (enade.NU_IDADE >= 20) & (enade.NU_IDADE <= 50)
].NT_GER.describe()

In [ ]:
# Tabela cruzada
# Contagem de aluno por sexo

enade.TP_SEXO.value_counts() / enade.shape[0]

In [ ]:
# Comparando médias de notas para cada região do Brasil
enade[['NT_GER', 'NT_FG', 'NT_CE']].describe()


In [ ]:
# Agregando por região do curso
enade.groupby('CO_REGIAO_CURSO').agg({
    "NT_GER" : 'mean',
    "NT_FG"  : 'mean',
    "NT_CE"  : 'mean'
})

## Transformações

CO_IES
CO_CATEGAD
CO_GRUPO
CO_MODALIDADE
CO_UF_CURSO
CO_REGIAO_CURSO
NU_IDADE
TP_SEXO
NT_GER
NT_FG
NT_CE

In [ ]:
# cria coluna DESC_PUBLICA e faz um filtro na CO_CATEGAD entre IES privadas e publicas
enade['DESC_PUBLICA'] = ""
enade.loc[
    enade.CO_CATEGAD.isin([118, 120, 121, 10005, 10006, 10007, 10008, 10009, 17634 ]),
    "DESC_ PUBLICA"
] = "Privado"

enade.loc[
    enade.CO_CATEGAD.isin([93, 115, 116, 10001, 10002, 10003]),
    "DESC_PUBLICA"

] = "Público"

In [ ]:
enade.DESC_PUBLICA.value_counts() / enade.shape[0]

In [ ]:
# transformando a modalidade. Obs.: apenas para demonstração. Dar replace
# nas variaveis não é a melhor prática. Para isso, criamos uma nova como exemplo
# DESC_PUBLICA

enade['CO_MODALIDADE'] = enade.CO_MODALIDADE.replace({
    0 : "EAD",
    1 : "Presencial"
    
})

In [ ]:
enade.CO_MODALIDADE.value_counts()

In [ ]:
# Transforma região

enade['CO_REGIAO_CURSO'] = enade.CO_REGIAO_CURSO.replace({
    1 : 'Norte',
    2 : 'Nordeste',
    3 : 'Sudeste',
    4 : 'Sul',
    5 : 'Centro Oeste'
})

In [ ]:
enade.CO_REGIAO_CURSO.value_counts()

In [ ]:
# Transformando cor/raça

enade.QE_I02.value_counts() #espaço em branco deve ser tratado

In [ ]:
enade["DESC_COR"] = enade.QE_I02.replace({
    'A' : 'Branca',
    'B' : 'Preta',
    'C' : 'Amarela',
    'D' : 'Parda',
    'E' : 'Indigena',
    'F' : pd.NA,      #tratamento para campos em brancos, assim o análise estatística não levará em conta esses campos
    ' ' : pd.NA
    
})

In [ ]:
dict(enade.DESC_COR.value_counts())

## Outras questões:

1 - Qual é a nota geral média dos alunos do Nordeste?

In [ ]:
enade.loc[  
    enade.CO_REGIAO_CURSO == "Nordeste",
    'NT_GER'

].mean()

2 - Qual é a média da nota do componente específico dos alunos do RS dos cursos de engenharia elétrica?

In [ ]:
enade.loc [
    (enade.CO_UF_CURSO == 43) &
    (enade.CO_GRUPO == 5806),
    'NT_CE'
].mean()

34.79692671394829

3 - Qual a média da nota do componete de formação geral de alunas (sexo feminino) da cor parda em minas gerais, em cursos presenciais de Engenharia de Produção?

In [ ]:
enade.loc[
    (enade.TP_SEXO == 'F') & 
    (enade.DESC_COR == 'Parda') & 
    (enade.CO_UF_CURSO == 31) &
    (enade.CO_MODALIDADE == 'Presencial') & 
    (enade.CO_GRUPO == 6208),
    "NT_FG"
].mean()

40.19142394822005